ref: https://www.kaggle.com/code/suchandramajumder/stock-news-sentiment-analysis

In [9]:
import pandas as pd
from textblob import TextBlob

In [10]:
df = pd.read_csv('~/Downloads/archive (1)/raw_partner_headlines.csv', sep=',', header=0)

In [11]:
df

,Unnamed: 0,headline,url,publisher,date,stock
0,2,Agilent Technologies Announces Pricing of $5……...,http://www.gurufocus.com/news/1153187/agilent-...,GuruFocus,2020-06-01 00:00:00,A
1,3,Agilent (A) Gears Up for Q2 Earnings: What's i...,http://www.zacks.com/stock/news/931205/agilent...,Zacks,2020-05-18 00:00:00,A
2,4,J.P. Morgan Asset Management Announces Liquida...,http://www.gurufocus.com/news/1138923/jp-morga...,GuruFocus,2020-05-15 00:00:00,A
3,5,"Pershing Square Capital Management, L.P. Buys ...",http://www.gurufocus.com/news/1138704/pershing...,GuruFocus,2020-05-15 00:00:00,A
4,6,Agilent Awards Trilogy Sciences with a Golden ...,http://www.gurufocus.com/news/1134012/agilent-...,GuruFocus,2020-05-12 00:00:00,A
...,...,...,...,...,...,...
1845554,1849874,Consumer Cyclical Sector Wrap,https://www.benzinga.com/content/12/08/2846030...,webmaster,2012-08-20 00:00:00,ZX
1845555,1849875,Consumer Cyclical Sector Wrap,https://www.benzinga.com/content/12/07/2767124...,webmaster,2012-07-23 00:00:00,ZX
1845556,1849876,Zacks #5 Rank Additions for Monday - Tale of t...,http://www.zacks.com/stock/news/73497/here-are...,Zacks,2012-04-23 00:00:00,ZX
1845557,1849877,4 Stock Strategies From Wall Street: Feb. 9 (U...,http://www.thestreet.com/story/11409053/1/4-st...,TheStreet.Com,2012-02-09 00:00:00,ZX


In [12]:
df=df.drop(['Unnamed: 0', 'publisher', 'url'] ,axis=1)

In [13]:
df

,headline,date,stock
0,Agilent Technologies Announces Pricing of $5……...,2020-06-01 00:00:00,A
1,Agilent (A) Gears Up for Q2 Earnings: What's i...,2020-05-18 00:00:00,A
2,J.P. Morgan Asset Management Announces Liquida...,2020-05-15 00:00:00,A
3,"Pershing Square Capital Management, L.P. Buys ...",2020-05-15 00:00:00,A
4,Agilent Awards Trilogy Sciences with a Golden ...,2020-05-12 00:00:00,A
...,...,...,...
1845554,Consumer Cyclical Sector Wrap,2012-08-20 00:00:00,ZX
1845555,Consumer Cyclical Sector Wrap,2012-07-23 00:00:00,ZX
1845556,Zacks #5 Rank Additions for Monday - Tale of t...,2012-04-23 00:00:00,ZX
1845557,4 Stock Strategies From Wall Street: Feb. 9 (U...,2012-02-09 00:00:00,ZX


In [14]:
df.isnull().sum()

headline    0
date        0
stock       0
dtype: int64

In [15]:
stock = df['stock'].unique()

In [16]:
stock.size

6552

In [17]:
sen = []
for text in df['headline']:
    blob = TextBlob(text)
    sentiment = blob.sentiment.polarity
    sen.append(sentiment)

In [18]:
df['sentiment score'] = sen

In [19]:
sen_st=[]
for i in stock:
    s=0
    data = df[(df['stock']==i)]
    s = data['sentiment score'].sum()
    sen_st.append(s)

In [22]:
dataframe = pd.DataFrame({'Stock': stock, 'Sentiment': sen_st})

In [21]:
dataframe.head()

,Stock,Sentiment
0,A,38.449404
1,AA,5.617893
2,AAC,11.082172
3,AADR,2.684962
4,AAL,1.212437


---

In [38]:
submissions = pd.read_csv('~/Downloads/submission.csv', sep=',', header=0)
submissions.set_index('time', inplace=True)
#submissions[submissions.index >= '2020-06-30']
submissions

,assetCode,confidenceValue
time,,
2017-01-03,A.N,0.015291
2017-01-03,AA.N,-0.062404
2017-01-03,AAL.O,-0.001005
2017-01-03,AAN.N,0.021894
2017-01-03,AAP.N,0.016215
...,...,...
2019-07-15,ZION.O,-0.026452
2019-07-15,ZNGA.O,-0.049116
2019-07-15,ZS.O,-0.185345


---
ref = [https://www.kaggle.com/code/equinxx/stock-prediction-gan-twitter-sentiment-analysis/notebook](https://www.kaggle.com/code/equinxx/stock-prediction-gan-twitter-sentiment-analysis/notebook)

In [46]:
import os
import numpy as np
import csv
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import math
import time
import tensorflow as tf
from tensorflow.keras.layers import GRU, LSTM, Bidirectional, Dense, Flatten, Conv1D, BatchNormalization, LeakyReLU, Dropout
from tensorflow.keras import Sequential
from keras.utils.vis_utils import plot_model
from pickle import load
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import statsmodels.api as sm
from math import sqrt
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from pickle import dump
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import unicodedata

import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'keras.utils.vis_utils'